In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib as plt

pd.options.display.float_format = "{:,.2f}".format

#### Load In Data

In [3]:
items = pd.read_excel(r'C:\Users\floral02\Documents\sklearn\PRUEBA Web scraper y category classifier.xlsx')

In [4]:
#Limpio nombres de columnas
items = items.rename(columns={"#PR LOC 1000001 : ITM_DESC": "Desc"})

In [5]:
#Guardo etiquetas de mis columnas a predecir
labels_SG = items['SUPER GROUP'].unique()
labels_PG = items['PRODUCT GROUP'].unique()

In [6]:
#Elimino nulos en mi training set. Las filas con descripciones vacías porque esa es mi X
items.dropna(subset = ["ITEM DESCRIPTIONS"], inplace=True)

#### Load Classifier

In [34]:
with open('./models/category_classifier.pkl', 'rb') as f:
    log = pickle.load(f)

with open('./models/category_classifier_z.pkl', 'rb') as f:
    log_z = pickle.load(f)

with open('./models/category_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
    
with open('./models/category_vectorizer_2.pkl', 'rb') as f:
    vectorizer_2 = pickle.load(f)

#### Predicting New Items

In [35]:
raw_test = pd.read_excel(r'C:\Users\floral02\Documents\sklearn\ITEMS PR - SIN CREAR.xlsx')

In [36]:
raw_test = raw_test.rename(columns={"ITEM DESCRIPTION": "Desc"})

In [37]:
raw_test_list = raw_test['Desc'].tolist()

In [38]:
raw_test_vectors = vectorizer.transform(raw_test_list)

In [39]:
raw_test_vectors_2 = vectorizer_2.transform(raw_test_list)

In [40]:
lr_test_pred = log.predict(raw_test_vectors_2)
lr_test_pred

lr_test_pred_2 = log_z.predict(raw_test_vectors_2)
lr_test_pred_2

df_predicted = pd.DataFrame({'Super Group Predicted':lr_test_pred})
df_predicted_2 = pd.DataFrame({'Product Group Predicted':lr_test_pred_2})

proba = log.predict_proba(raw_test_vectors_2)
proba = pd.DataFrame([max(x) for x in proba], columns = ['Proba Super Group'])

proba_2 = log_z.predict_proba(raw_test_vectors_2)
proba_2 = pd.DataFrame([max(x) for x in proba_2], columns = ['Proba Product Group'])

csv = pd.concat([raw_test.reset_index(drop=False), df_predicted.reset_index(drop=True), 
                   df_predicted_2.reset_index(drop=True), proba.reset_index(drop=True), 
                proba_2.reset_index(drop=True)], axis=1)

csv.to_csv(r'C:\Users\floral02\Documents\sklearn\predictions_SG_PG_con_proba.csv', index = False, header=True)